In [30]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import tqdm


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [6]:
answers = {}

In [7]:
# Some data structures that will be useful

In [8]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [9]:
len(allRatings)

200000

In [107]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in allRatings:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [11]:
##################################################
# Read prediction                                #
##################################################

In [12]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [13]:
# in[12] 定义负采样函数
def sample_negative(user, positive_books, all_books, num_samples=1):
    negative_samples = []
    while len(negative_samples) < num_samples:
        neg_book = random.choice(all_books)
        if neg_book not in positive_books:
            negative_samples.append(neg_book)
    return negative_samples

In [131]:
# in[13] 构建验证集
validation_positive = ratingsValid
validation_negative = []
all_books = list(bookCount.keys())

user_positive_books = defaultdict(set)
for u, b, r in allRatings:
    user_positive_books[u].add(b)

for u, b, r in validation_positive:
    negative_books = sample_negative(u, user_positive_books[u], all_books)
    for neg_b in negative_books:
        validation_negative.append((u, neg_b, 0))

In [132]:
# 合并正负样本
validation_set = [(u, b, 1) for u, b, r in validation_positive] + validation_negative

# in[14] 评估基线模型的准确率
def evaluate_baseline(validation_set, mostPopular_set):
    correct = 0
    for u, b, label in validation_set:
        prediction = 1 if b in mostPopular_set else 0
        if prediction == label:
            correct += 1
    accuracy = correct / len(validation_set)
    return accuracy

acc1 = evaluate_baseline(validation_set, return1)
answers['Q1'] = acc1

In [125]:
### Question 1

In [133]:
answers['Q1'] = acc1

In [134]:
print(answers['Q1'])

0.7547


In [128]:
assertFloat(answers['Q1'])

In [19]:
### Question 2

In [135]:
# in[16] 寻找最佳阈值
def find_best_threshold(book_counts, total_read, validation_set):
    thresholds = [i*0.01 for i in range(1,100)]
    best_acc = 0
    best_threshold = 0
    for threshold in tqdm.tqdm(thresholds):
        return1 = set()
        count = 0
        for ic, i in mostPopular:
            count += ic
            return1.add(i)
            if count > totalRead * threshold:
                break
        acc = evaluate_baseline(validation_set, return1)
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold
    return best_threshold, best_acc

threshold, acc2 = find_best_threshold(bookCount, totalRead, validation_set)
answers['Q2'] = [threshold, acc2]

 12%|█▏        | 815/6688 [00:04<00:31, 187.09it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/root/miniconda3/envs/torch/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_8444/2184903451.py", line 14, in <module>
    threshold, acc2 = find_best_threshold(bookCount, totalRead, validation_set)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_8444/2184903451.py", line 8, in find_best_threshold
    acc = evaluate_baseline(validation_set, popular_set)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_8444/4151642438.py", line None, in evaluate_baseline
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/miniconda3/envs/torch/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceb

In [21]:
print(answers['Q2'])

[35, 0.75145]


In [109]:
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead*threshold: break

In [23]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

In [24]:
### Question 3/4

In [25]:
def jaccard_similarity(item1, item2, ratingsPerItem):
    users1 = set([u for u, r in ratingsPerItem[item1]])
    users2 = set([u for u, r in ratingsPerItem[item2]])
    intersection = users1 & users2
    union = users1 | users2
    if not union:
        return 0
    return len(intersection) / len(union)

In [26]:
def evaluate_jaccard(validation_set, ratingsPerUser, ratingsPerItem, threshold):
    correct = 0
    for u, b, label in validation_set:
        read_items = [item for item, r in ratingsPerUser[u]]
        similarities = [jaccard_similarity(b, b_prime, ratingsPerItem) for b_prime in read_items]
        max_similarity = max(similarities) if similarities else 0
        prediction = 1 if max_similarity > threshold else 0
        if prediction == label:
            correct += 1
    accuracy = correct / len(validation_set)
    return accuracy

In [35]:
def find_best_jaccard_threshold(validation_set, ratingsPerUser, ratingsPerItem):
    thresholds = [i * 0.0001 for i in range(0, 21)]
    best_acc = 0
    best_threshold = 0
    for threshold in tqdm.tqdm(thresholds):
        acc = evaluate_jaccard(validation_set, ratingsPerUser, ratingsPerItem, threshold)
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold
    return best_threshold, best_acc

threshold_jaccard, acc3 = find_best_jaccard_threshold(validation_set, ratingsPerUser, ratingsPerItem)
answers['Q3'] = acc3

100%|██████████| 21/21 [01:52<00:00,  5.37s/it]


In [36]:
print(threshold_jaccard,acc3)

0.0019 0.69435


In [119]:
# in[24] 结合 Jaccard 和流行度阈值
def evaluate_combined(validation_set, mostPopular_set, ratingsPerUser, ratingsPerItem, jaccard_threshold):
    correct = 0
    for u, b, label in validation_set:
        popularity_pred = 1 if b in mostPopular_set else 0
        read_items = [item for item, r in ratingsPerUser[u]]
        similarities = [jaccard_similarity(b, b_prime, ratingsPerItem) for b_prime in read_items]
        if similarities:
            jaccard_pred = 1 if max(similarities) > jaccard_threshold else 0
        else:
            jaccard_pred = 0
        prediction = popularity_pred | jaccard_pred
        if prediction == label:
            correct += 1
    accuracy = correct / len(validation_set)
    return accuracy

# 找到最佳组合阈值
def find_best_combined_thresholds(validation_set, mostPopular, ratingsPerUser, ratingsPerItem, totalRead):
    thresholds = [i*0.01 for i in range(1,100)]  # 避免 threshold 为 0
    best_acc = 0
    best_threshold = 0
    best_jaccard_threshold = 0
    for threshold in tqdm.tqdm(thresholds, desc="Optimizing threshold"):
        # 根据当前 threshold 生成 return1
        return1 = set()
        count = 0
        for ic, i in mostPopular:
            count += ic
            return1.add(i)
            if count > totalRead * threshold:
                break
        j_threshold = 0.019
        acc = evaluate_combined(validation_set, return1, ratingsPerUser, ratingsPerItem, j_threshold)
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold
            best_jaccard_threshold = j_threshold
    return best_threshold, best_jaccard_threshold, best_acc

best_threshold, best_jaccard_threshold, best_acc = find_best_combined_thresholds(validation_set, 
                                                         mostPopular, 
                                                         ratingsPerUser, 
                                                         ratingsPerItem, 
                                                         totalRead)


Optimizing threshold: 100%|██████████| 99/99 [13:22<00:00,  8.11s/it]


In [120]:
print(best_threshold,best_jaccard_threshold,best_acc)

0.01 0.019 0.9819


In [44]:
answers['Q3'] = acc3
answers['Q4'] = acc4

In [45]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [122]:
# in[26] 生成 predictions_Read.csv
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead * 0.7:
        break
with open("predictions_Read.csv", 'w') as predictions:
    for l in open("pairs_Read.csv"):
        if l.startswith("userID"):
            predictions.write(l)
            continue
        u, b = l.strip().split(',')
        # 使用结合后的模型进行预测
        popularity_pred = 1 if b in return1 else 0
        read_items = [item for item, r in ratingsPerUser[u]]
        similarities = [jaccard_similarity(b, b_prime, ratingsPerItem) for b_prime in read_items]
        if similarities:
            jaccard_pred = 1 if max(similarities) > 0.09 else 0
        else:
            jaccard_pred = 0
        prediction = 1 if max(popularity_pred, jaccard_pred) else 0
        predictions.write(f"{u},{b},{prediction}\n")

# 将答案提交到 Gradescope
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [47]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [48]:
assert type(answers['Q5']) == str

In [49]:
##################################################
# Rating prediction                              #
##################################################

In [96]:
from surprise import SVDpp, Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy
import pandas as pd

# 加载数据到 DataFrame
df = pd.DataFrame(allRatings, columns=['userId', 'bookId', 'rating'])

# 定义 Reader
reader = Reader(rating_scale=(1, 5))

# 加载数据到 Surprise 的 Dataset
data = Dataset.load_from_df(df[['userId', 'bookId', 'rating']], reader)

# 定义参数搜索空间
param_grid = {
    'n_epochs': [5,10,15,20],          # 训练轮数
    'lr_all': [0.008],   # 学习率
    'reg_all': [0.1],      # 正则化参数
    'n_factors': [50]        # 潜在因子数量
}

# 使用 GridSearchCV 进行超参数调优
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3, joblib_verbose=1)
gs.fit(data)

# 输出最佳参数和对应的 RMSE
print("最佳参数组合:")
print(gs.best_params['rmse'])
print(f"最佳 RMSE: {gs.best_score['rmse']:.4f}")

# 使用最佳参数训练模型
best_params = gs.best_params['rmse']
model = SVDpp(
    n_epochs=20,
    lr_all=0.011,
    reg_all=0.1,
    n_factors=50
)

# 划分训练集和测试集
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
full_trainset = data.build_full_trainset()
# 训练模型
model.fit(full_trainset)

# 进行预测
predictions = model.test(testset)

# 计算并输出 RMSE
rmse = accuracy.rmse(predictions)
print(f"验证集上的 RMSE: {rmse:.4f}")

# 将最佳参数和 RMSE 保存到答案字典
answers['Q8_SVDpp'] = {
    '最佳参数': best_params,
    '最佳 RMSE': gs.best_score['rmse']
}
answers['Q9_SVDpp'] = "I confirm that I have uploaded an assignment submission to gradescope"

最佳参数组合:
{'n_epochs': 20, 'lr_all': 0.008, 'reg_all': 0.1, 'n_factors': 50}
最佳 RMSE: 1.2196


KeyboardInterrupt: 

In [99]:

model = SVDpp(
    n_epochs=20,
    lr_all=0.007,
    reg_all=0.1,
    n_factors=50
)
full_trainset = data.build_full_trainset()
# 训练模型
model.fit(full_trainset)
# in[49] 生成 predictions_Rating_SVD.csv
with open("predictions_Rating.csv", 'w') as predictions:
    for l in open("pairs_Rating.csv"):
        if l.startswith("userID"):
            predictions.write(l)
            continue
        u, b = l.strip().split(',')
        pred = model.predict(u, b).est

        predictions.write(f"{u},{b},{pred}\n")

#1.437 20 0.008 0.1 50
#1.448 20 0.011 0.1 50
#1.437 20 0.007 0.1 50

In [105]:
from surprise import SVD
model = SVD(
    n_epochs=20,
    lr_all=0.007,
    reg_all=0.1,
    n_factors=20
)
full_trainset = data.build_full_trainset()
# 训练模型
model.fit(full_trainset)
# in[49] 生成 predictions_Rating_SVD.csv
with open("predictions_Rating.csv", 'w') as predictions:
    for l in open("pairs_Rating.csv"):
        if l.startswith("userID"):
            predictions.write(l)
            continue
        u, b = l.strip().split(',')
        pred = model.predict(u, b).est

        predictions.write(f"{u},{b},{pred}\n")

#1.430 20 0.007 0.1 50
#1.44 20 0.007 0.1 200
#1.426 20 0.007 0.1 40
#1.4249 20 0.007 0.1 20

In [104]:
param_grid = {
    'n_epochs': [20],          # 训练轮数
    'lr_all': [0.008],   # 学习率
    'reg_all': [0.1],      # 正则化参数
    'n_factors': [20, 30,35, 40,45]        # 潜在因子数量
}

# 使用 GridSearchCV 进行超参数调优
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, joblib_verbose=1)
gs.fit(data)

# 输出最佳参数和对应的 RMSE
print("最佳参数组合:")
print(gs.best_params['rmse'])
print(f"最佳 RMSE: {gs.best_score['rmse']:.4f}")

# 使用最佳参数训练模型
best_params = gs.best_params['rmse']
model = SVD(
    n_epochs=best_params['n_epochs'],
    lr_all=best_params['lr_all'],
    reg_all=best_params['reg_all'],
    n_factors=best_params['n_factors']
)

最佳参数组合:
{'n_epochs': 20, 'lr_all': 0.008, 'reg_all': 0.1, 'n_factors': 20}
最佳 RMSE: 1.2148


In [141]:
# 第一轮：在20附近微调n_factors
param_grid = {
    'n_epochs': [15, 18, 20, 22, 25],          
    'lr_all': [0.008],   # 在0.007附近搜索
    'reg_all': [0.12],      
    'n_factors': [20]  # 使用第一轮找到的最佳n_factors
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, joblib_verbose=1)
gs.fit(data)
best_params = gs.best_params['rmse']
print(best_params)

{'n_epochs': 22, 'lr_all': 0.008, 'reg_all': 0.12, 'n_factors': 20}


In [143]:
print(f"最佳 RMSE: {gs.best_score['rmse']:.4f}")

最佳 RMSE: 1.2145


In [150]:
from surprise import SVD
model = SVD(
    n_epochs=22,
    lr_all=0.008,
    reg_all=0.11,
    n_factors=23
)
full_trainset = data.build_full_trainset()
# 训练模型
model.fit(full_trainset)
# in[49] 生成 predictions_Rating_SVD.csv
with open("predictions_Rating.csv", 'w') as predictions:
    for l in open("pairs_Rating.csv"):
        if l.startswith("userID"):
            predictions.write(l)
            continue
        u, b = l.strip().split(',')
        pred = model.predict(u, b).est

        predictions.write(f"{u},{b},{pred}\n")

#1.430 20 0.007 0.1 50
#1.44 20 0.007 0.1 200
#1.426 20 0.007 0.1 40
#1.4249 20 0.007 0.1 20
#1.4264 20 0.008 0.12 22
#1.42018 22 0.008 0.12 22
#1.42196 22 0.008 0.13 22
#1.4182 22 0.008 0.11 22
#1.43